In [ ]:
# Import dependencies
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D
from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
import gc
import pandas as pd
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from sklearn.model_selection import train_test_split

In [ ]:
# Mount to Drive
from google.colab import drive
drive.mount('/content/drive')
# Set working directory
%cd /content/drive/My Drive/Emotion_detection_data

# Declare dataset path
dataset_folder='./input/CK+48'
sub_folders=os.listdir(dataset_folder)

# Read folder names as labels and images underneath
i=0
last=[]
images=[]
labels=[]
temp = sub_folders

# Read folders in the main dataset folder, one at a time
for sub_folder in sub_folders:
  sub_folder_index = temp.index(sub_folder)
  label = sub_folder_index
  # Define labels basis use case. We are using positive:0, negative:1, neutral:2
  # for our use case of predicting emotions of visitors entering a retail store
  if  label in [4, 6]:    # label in ['happy', 'surprise']
    new_label=0           # changed to label = positive emotion
  elif label in [0,5]:    # label in ['anger','sadness']
    new_label=1           # changed to label = negative emotion
  else:                   # label in ['contempt', 'disgust', 'fear']
    new_label=2           # changed to label = neutral emotion

  path = dataset_folder+'/'+sub_folder
  sub_folder_images= os.listdir(path)

  # Read images in the sub folder, one at a time
  for image in sub_folder_images:
    image_path = path+'/'+image
    image = cv2.imread(image_path)
    images.append(image)
    labels.append(new_label)
    i+=1
  last.append(i)


MessageError: ignored

In [ ]:
def padding(image, target_height, target_width, num_channels):
    height, width, channels = image.shape

    if height >= target_height and width >= target_width:
        # If the image is larger or equal to the target size, no need to pad
        return image

    pad_height = max(target_height - height, 0)
    pad_width = max(target_width - width, 0)

    top_pad = pad_height // 2
    bottom_pad = pad_height - top_pad
    left_pad = pad_width // 2
    right_pad = pad_width - left_pad

    padded_image = np.pad(image, ((top_pad, bottom_pad), (left_pad, right_pad), (0, 0)), mode='constant')
    return padded_image



In [ ]:
# Declare x and y
images_x = np.array(images)
labels_y = np.array(labels)

# Normalizing the data
images_x = images_x/255

# encoding the labels
num_of_classes = 3 # positive, negative, neutral
labels_y_encoded = tf.keras.utils.to_categorical(labels_y,num_classes=num_of_classes)

# Split data into 70:15:15 train, val, and test
X_train, X_test, y_train, y_test = train_test_split(images_x, labels_y_encoded, random_state=54, test_size=0.30, shuffle=True)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=10)

In [ ]:
images_x[0].shape

(48, 48, 3)

In [ ]:
# # pretrained Inceptionv3 Model

# inputs = tf.keras.Input(shape=(48, 48, 3))

# base_model = tf.keras.applications.VGG16(
#     input_tensor=inputs,
#     include_top = False,
#     weights = 'imagenet'
# )

# base_model.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# hidden = tf.keras.layers.Dense(2048, activation = 'relu')(base_model.layers[-2].output)
# hidden = tf.keras.layers.Dense(1564, activation = 'relu')(hidden)
# hidden = tf.keras.layers.Dense(1001, activation = 'relu')(hidden)
# hidden = tf.keras.layers.Dense(2048, activation = 'relu')(hidden)
# output = tf.keras.layers.Dense(3, activation = 'softmax')(hidden)

# model = tf.keras.Model(inputs = inputs, outputs = output)
# model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0   

In [ ]:
# CNN Architecture
input = Input(shape = (48,48,3))
conv1 = Conv2D(100,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(input)
conv1 = Activation('relu')(conv1)
conv1 = Activation('relu')(conv1)
pool1 = MaxPooling2D(pool_size = (2,2)) (conv1)
conv2 = Conv2D(150,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool1)
conv2 = Activation('relu')(conv2)
pool2 = MaxPooling2D(pool_size = (2,2), padding='same') (conv2)
conv3 = Conv2D(128,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool2)
conv3 = Dropout(0.3)(conv3)
conv3 = Activation('relu')(conv3)
pool3 = MaxPooling2D(pool_size = (2,2), padding='same') (conv3)
conv4 = Conv2D(256,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool3)
conv4 = Activation('relu')(conv4)
pool4 = MaxPooling2D(pool_size = (2,2), padding='same') (conv4)
layer4 = Dense(1000, activation=tf.nn.relu) (pool4)
drop_1 = Flatten()(layer4)
dense_1 = Dense(100,activation='relu')(drop_1)
output = Dense(3,activation="softmax")(dense_1)

# Model compile
model = Model(inputs=input,outputs=output)

In [ ]:
loss_func = tf.keras.losses.categorical_crossentropy

# Use the RMSprop learning algorithm to optimize the network weights
base_learning_rate = 0.0001
opt = tf.keras.optimizers.Adam(lr=base_learning_rate)

# Compile the model using the specified loss function and potimizer
model.compile(loss=loss_func, optimizer=opt, metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 48, 48, 100)       2800      
                                                                 
 activation (Activation)     (None, 48, 48, 100)       0         
                                                                 
 activation_1 (Activation)   (None, 48, 48, 100)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 100)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 150)       135150    
                                                             

In [ ]:
class garbage_collect_callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    gc.collect()

history = model.fit(x = X_train, # Train examples
          y = y_train, # Train labels
          epochs = 30, # number of epochs (passes through data during training)
          batch_size = 50, # number of points to consider in each optimizer iteration
          callbacks = [garbage_collect_callback()],
          validation_data = (X_val, y_val), #data to use for validation
          verbose=1) #will print information about optimization process

Epoch 1/30
15/15 [==============================] - 29s 2s/step - loss: 1.4817 - accuracy: 0.5522 - val_loss: 1.1631 - val_accuracy: 0.7006
Epoch 2/30
15/15 [==============================] - 19s 1s/step - loss: 1.1689 - accuracy: 0.6126 - val_loss: 1.0820 - val_accuracy: 0.7006
Epoch 3/30
15/15 [==============================] - 18s 1s/step - loss: 1.0978 - accuracy: 0.6126 - val_loss: 1.0342 - val_accuracy: 0.7006
Epoch 4/30
15/15 [==============================] - 21s 1s/step - loss: 1.0619 - accuracy: 0.6126 - val_loss: 1.0165 - val_accuracy: 0.7006
Epoch 5/30
15/15 [==============================] - 18s 1s/step - loss: 1.0369 - accuracy: 0.6126 - val_loss: 0.9789 - val_accuracy: 0.7006
Epoch 6/30
15/15 [==============================] - 18s 1s/step - loss: 1.0160 - accuracy: 0.6126 - val_loss: 0.9364 - val_accuracy: 0.7006
Epoch 7/30
15/15 [==============================] - 19s 1s/step - loss: 0.9834 - accuracy: 0.6154 - val_loss: 1.0062 - val_accuracy: 0.7006
Epoch 8/30
15/15 [==

In [ ]:
test_pred = model.predict(X_test)
scores = model.evaluate(X_test, y_test, verbose = 0)
print('\nTesting model on mock_test set:')
print(f'Model Loss: {scores[0]:.3f}, Model Accuracy: {scores[1]:.3f}')

5/5 [==============================] - 1s 171ms/step

Testing model on mock_test set:
Model Loss: 0.129, Model Accuracy: 0.981
